# RUN

In [1]:
import os
import pandas as pd
from modules.data_pipeline import process_data, get_path, export_file

export_file.export_to_csv('June', 'igv')

# dict = process_data.read(21, 'igv')
# paths = get_path.get(21,'igv')
# paths

Now processing ---> ('ICA', 'ica20240601', 'AT02Data.csv')
Now processing ---> ('ICA', 'ica20240601', 'AT03Data.csv')
Now processing ---> ('ICA', 'ica20240601', 'AT06Data (1).csv')
Now processing ---> ('ICA', 'ica20240602', 'AT02Data (1).csv')
Now processing ---> ('ICA', 'ica20240602', 'AT03Data (1).csv')
Now processing ---> ('ICA', 'ica20240604', 'AT01Data (1).csv')
Now processing ---> ('ICA', 'ica20240604', 'AT02Data (1).csv')
Now processing ---> ('ICA', 'ica20240604', 'AT03Data (2).csv')
Now processing ---> ('ICA', 'ica20240604', 'AT06Data (1).csv')
Now processing ---> ('ICA', 'ica20240605', 'AT02Data.csv')
Now processing ---> ('ICA', 'ica20240605', 'AT03Data.csv')
Now processing ---> ('ICA', 'ica20240606', 'AT02Data (3).csv')
Now processing ---> ('ICA', 'ica20240606', 'AT03Data (4).csv')
Now processing ---> ('ICA', 'ica20240606', 'AT06Data (2).csv')
Now processing ---> ('ICA', 'ica20240608', 'AT01Data (2).csv')
Now processing ---> ('ICA', 'ica20240608', 'AT01Data.csv')
Now processi

# Test all weekly file

In [ ]:
import os
import pandas as pd
import numpy as np

def get_all_weekly_file():
    root = 'c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99'
    pathlist = []
    for item in os.listdir(root):
        p = os.path.join(root, item)
        if os.path.isfile(p):
            pathlist.append(p)
    return pathlist

def lmd_rtg_operation(location_ref=str, task_stage=str):
    if ('YARD' in location_ref) & (task_stage=='Waiting for Operation'):
        return 'Waiting for Operation'
    else:
        return np.NaN

li = get_all_weekly_file()

In [ ]:
for p in li:
    print(p)

    df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)

    print(df.columns)
    print()

In [ ]:
df.columns


In [ ]:
df['task_stage'].value_counts().index.tolist()

In [ ]:
path = os.path.abspath('c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W17\Igv_W17_TPY.csv')

d = pd.read_csv(path, encoding='utf-8-sig')
d.columns

In [ ]:
def lambda_power_usage(soc_diff):
    if soc_diff > 0 and soc_diff<=5: return soc_diff
    elif soc_diff<=0 and soc_diff>-5: return 0
    elif soc_diff >5: return 5
    elif soc_diff<=-5: return -1


for vehicle, d2 in d.groupby('vehicle_id'):
    for cycle, data in d2.groupby('Cycle Tag'):
        data['soc_diff'] = data['soc'] - data['soc'].shift(-1)
        data['power_usage'] = data['soc_diff'].apply(lambda x: lambda_power_usage(x))
        print( data['soc_diff'].value_counts().index.tolist(),
              data['power_usage'].value_counts().index.tolist())
        print()

# CKP info extract

In [ ]:
import pandas as pd
# display('C:\\Users\westwell\Documents\\.WorkDocuments\github\westwell_da\data\processed\W18\ICA\IgvData_ica20240504.csv')

p = 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\WMay\\ICA\\IgvData_ica20240503.csv'

d = pd.read_csv(p, index_col=0)
display(d.columns)

In [ ]:
def feature_extraction(group):
    duration = group['duration_min'].sum()  # Example feature: sum of duration
    vehicle  = group['vehicle_id'].min()
    task_type= group['mission_type_org'].min()
    start    = group['local_time'].min()
    end      = group['local_time'].max()
    ck1      = group['local_time']
    missionID= group['missionID'].value_counts().index.tolist()
    loc      = group['target_location'].value_counts().index.tolist()

    return pd.Series({'duration_min': duration, 
                      'AT ID': vehicle,
                      'mission_type': task_type,
                      'cycle_start':start,
                      'cycle_end':  end,
                      'missionID':missionID,
                      'target_location':loc
                      })


grouped = d.groupby('Cycle Tag').apply(feature_extraction)

display(grouped.head(1))

# Group by 'Cycle Tag' and 'ckp_demo' and find the min of 'local_time'
g = d.groupby(['Cycle Tag', 'ckp_demo'])['local_time'].min().reset_index()

# Pivot the DataFrame
pivoted = g.pivot(index='Cycle Tag', columns='ckp_demo', values='local_time').reset_index()
display(pivoted.head(1))

merged = pd.merge(grouped, pivoted, on='Cycle Tag', how='left')
merged.to_csv('ckp_info_export_0503.csv',date_format="%Y-%m-%d %H:%M:%S")
display(merged)

### TS IGV Data

In [7]:
import pandas as pd

p = "C:\\Users\westwell\Documents\\.WorkDocuments\github\westwell_da\data\\raw\W21\TS20240521\\561_VehicleData_TS.csv"

df = pd.read_csv(p)

display(df.columns)
display(df.dtypes)
display(df.describe().T)

Index(['vehicle_id', '   position_x', '   position_y', '   position_z',
       '   heading', '   speed', 'speed_command', '   mileage_distance',
       '   soc', '   local_time', '   chassis_mode', '   estop',
       '   charge_status', '   task_state_running', '   task_state_estop',
       '   location_type', '   task_stage', '   current_task', '   error_code',
       '   target_location', 'vesselVisitID', 'mission_type',
       ' container1_type', 'container2_type', 'vehicle_mode', 'missionID',
       'lane_change_state '],
      dtype='object')

vehicle_id                 int64
   position_x            float64
   position_y            float64
   position_z            float64
   heading               float64
   speed                 float64
speed_command            float64
   mileage_distance      float64
   soc                     int64
   local_time             object
   chassis_mode            int64
   estop                  object
   charge_status           int64
   task_state_running     object
   task_state_estop       object
   location_type          object
   task_stage             object
   current_task           object
   error_code             object
   target_location        object
vesselVisitID             object
mission_type              object
 container1_type           int64
container2_type            int64
vehicle_mode               int64
missionID                 object
lane_change_state          int64
dtype: object

,count,mean,std,min,25%,50%,75%,max
vehicle_id,19140.0,561.000000,0.000000,561.000000,561.000000,561.000000,561.000000,561.000000
position_x,19140.0,485.406815,168.788431,223.916791,289.952121,542.180226,601.821053,930.948729
position_y,19140.0,369.228812,163.440762,127.738534,169.383879,379.222204,557.884087,605.705940
position_z,19140.0,0.010623,0.011863,-0.035511,0.007401,0.012050,0.016841,0.056799
heading,19140.0,-0.817282,1.517319,-3.141585,-1.571648,-1.547658,0.002044,3.141587
speed,19140.0,0.312242,0.868550,-2.860888,-0.000695,0.000321,0.001819,5.463325
speed_command,19140.0,0.312967,0.868966,-2.968026,0.000000,0.000000,0.000000,5.398819
mileage_distance,19140.0,608.080408,6.671489,601.000000,601.000000,606.000000,615.000000,620.000000
soc,19140.0,59.960972,9.389956,44.000000,50.000000,63.000000,70.000000,71.000000
chassis_mode,19140.0,1.005643,0.076971,1.000000,1.000000,1.000000,1.000000,4.000000


In [1]:
pip install 'apache-airflow==2.6.1' --constraint "https://raw.githubusercontent.com/apache/airflow/constraints-2.6.1/constraints-3.7.txt"

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'apache-airflow==2.6.1'"


In [1]:
pip uninstall apache-airflow

In [1]:
import pandas as pd
import random
import csv
from datetime import datetime, timedelta

# Define the constraints and values
project_titles = ['TJ', 'WH', 'CK', 'ICAVE', 'AQCT', 'TS', 'TPY']
roles = ['IT', 'PM', 'QA']
milestones = ['Initiation', 'Planning', 'Execution', 'Monitoring', 'Closure']

# Generate projects
num_projects = 10
projects = []

for i in range(num_projects):
    project_code = f'P{i:03d}'
    title = random.choice(project_titles)
    milestone = random.choice(milestones)
    created_at = datetime.now() - timedelta(days=random.randint(1, 365))
    
    projects.append({
        'project_code': project_code,
        'title': title,
        'milestone': milestone,
        'created_at': created_at
    })

# Generate employees for each project
employees = []
employee_id = 1

for project in projects:
    project_code = project['project_code']
    
    # Ensure at least one PM
    employees.append({
        'id': employee_id,
        'role': 'PM',
        'working_hour': round(random.uniform(0.5, 9), 1),
        'project_code': project_code
    })
    employee_id += 1
    
    # Add IT roles (1 or 2)
    num_its = random.randint(1, 2)
    for _ in range(num_its):
        employees.append({
            'id': employee_id,
            'role': 'IT',
            'working_hour': round(random.uniform(0.5, 9), 1),
            'project_code': project_code
        })
        employee_id += 1
    
    # Add QA roles (1 to 4)
    num_qas = random.randint(1, 4)
    for _ in range(num_qas):
        employees.append({
            'id': employee_id,
            'role': 'QA',
            'working_hour': round(random.uniform(0.5, 9), 1),
            'project_code': project_code
        })
        employee_id += 1

# Convert to DataFrames
df_projects = pd.DataFrame(projects)
df_employees = pd.DataFrame(employees)

# Export to CSV
df_projects.to_csv(r'C:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\projects.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)
df_employees.to_csv(r'C:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\employees.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)
